<a href="https://colab.research.google.com/github/hamzafarooq/building-llm-applications-from-scratch/blob/main/Module%201/Module_1_Foundation_LLM_Maven_v2_module_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install nltk --quiet

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted
ERROR: Could not install packages due to an OSError: [Errno 1] Operation not permitted

You should consider upgrading via the '/Users/thekarteek/code/course/stanford/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
# code courtesy of https://nlpforhackers.io/language-models/
# Import necessary libraries
import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict


# Download required NLTK data
nltk.download('reuters',download_dir='/Users/thekarteek/code/course/course-llm-stanford/Module 1/')
nltk.download('punkt',download_dir='/Users/thekarteek/code/course/course-llm-stanford/Module 1/')

# Create a placeholder for the language model
# Using nested defaultdict to automatically handle new keys
model = defaultdict(lambda: defaultdict(lambda: 0))

# Build the trigram model
for sentence in reuters.sents():
    # Generate trigrams from each sentence
    # pad_right and pad_left add None at the beginning and end of the sentence
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        # Increment the count for this trigram
        model[(w1, w2)][w3] += 1

# Convert frequency counts to probabilities
for w1_w2 in model:
    # Calculate total count for this bigram
    total_count = float(sum(model[w1_w2].values()))

    # Convert each count to a probability
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

# At this point, model contains the probabilities of each word (w3)
# given the previous two words (w1, w2)

[nltk_data] Downloading package reuters to
[nltk_data]     /Users/thekarteek/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thekarteek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ValueError: No sentence tokenizer for this corpus

This code creates a trigram language model using the Reuters corpus. Here's a breakdown of what it does:

*   It imports necessary libraries and downloads required NLTK data.
*   It creates a nested defaultdict to store the model. This allows for easy counting of trigrams without explicitly checking if a key exists.
*   It iterates through all sentences in the Reuters corpus, generating trigrams for each sentence.
*   For each trigram, it increments the count in the model.
*   After counting all trigrams, it converts the counts to probabilities by dividing each count by the total count for its corresponding bigram.


The resulting model can be used to predict the probability of a word given the two preceding words.

In [ ]:
# Get the probabilities for words following "the price" and sort them
# This creates a list of (word, probability) tuples, sorted by probability in descending order
sorted_probabilities = sorted(dict(model["the", "price"]).items(), key=lambda x: x[1], reverse=True)

# Print the sorted probabilities
print("Most probable words following 'the price', in order:")
for word, prob in sorted_probabilities:
    print(f"{word}: {prob}")

Most probable words following 'the price', in order:
of: 0.3209302325581395
it: 0.05581395348837209
to: 0.05581395348837209
for: 0.05116279069767442
.: 0.023255813953488372
at: 0.023255813953488372
adjustment: 0.023255813953488372
is: 0.018604651162790697
,: 0.018604651162790697
paid: 0.013953488372093023
increases: 0.013953488372093023
per: 0.013953488372093023
the: 0.013953488372093023
will: 0.013953488372093023
cut: 0.009302325581395349
cuts: 0.009302325581395349
(: 0.009302325581395349
differentials: 0.009302325581395349
has: 0.009302325581395349
stayed: 0.009302325581395349
was: 0.009302325581395349
freeze: 0.009302325581395349
increase: 0.009302325581395349
would: 0.009302325581395349
yesterday: 0.004651162790697674
effect: 0.004651162790697674
used: 0.004651162790697674
climate: 0.004651162790697674
reductions: 0.004651162790697674
limit: 0.004651162790697674
now: 0.004651162790697674
moved: 0.004651162790697674
adjustments: 0.004651162790697674
slumped: 0.004651162790697674
mov

In [ ]:
!pip install pytorch-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
# Import required libraries
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


100%|██████████| 456318/456318 [00:00<00:00, 5700107.25B/s]


In [ ]:
# Define the input text
text = "I am thinking"
print(f"Input text: {text}")

# Encode the input text
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens to a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
model.eval()

# Check if CUDA is available and move model and tensors to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokens_tensor = tokens_tensor.to(device)
model.to(device)

print(f"Using device: {device}")




Input text: I am thinking


100%|██████████| 548118077/548118077 [00:19<00:00, 28242813.65B/s]


In [ ]:
# Predict next token
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# Get the predicted next sub-word (token)
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_token = tokenizer.decode([predicted_index])

# Add the predicted token to the original text
predicted_text = text + predicted_token

# Print the results
print(f"Predicted next token: '{predicted_token}'")
print(f"Complete predicted text: '{predicted_text}'")

#Assignment -- Extend the code above to a sentence

# Solution


In [ ]:
# Optional: Generate multiple next tokens
num_tokens_to_generate = 10
generated_text = text

for _ in range(num_tokens_to_generate):
    # Encode all text generated so far
    indexed_tokens = tokenizer.encode(generated_text)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)

    # Predict next token
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # Get the predicted next token
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    predicted_token = tokenizer.decode([predicted_index])

    # Add the predicted token to the generated text
    generated_text += predicted_token

print(f"\nGenerated text with {num_tokens_to_generate} additional tokens:")
print(generated_text)